# Simple Python HTTP(S) Server With GET/POST 
----
* `help(SimpleHTTPServer)`
* Standard Python library has a built-in module that can be used as minimalistic HTTP/HTTPS web server. It provides support of the protocol and allows you to extend capabilities by subclassing.
* Serve static HTML/CSS files to outside world can be very helpful and handy in many real life situations. For example, to show a client HTML pages you’ve created or stub an API by creating a static file.

## Example of static HTTP web server
----
* create a dummy API by creating json or/and xml files
* structure of resources organized in sub-folders will provide RESTful-like URLs. E.g. `/users/all.json`. 
* download data from a remote server. Let’s say there are some difficulties with `scp` command. It is possible to run simple server on the remote machine and download necessary contents via HTTP

In [ ]:
!python3 -m http.server 8000 --bind 127.0.0.1 
# This will be equavalient to what's below

In [ ]:
import SimpleHTTPServer
import SocketServer

PORT = 8000

Handler = SimpleHTTPServer.SimpleHTTPRequestHandler

httpd = SocketServer.TCPServer(("", PORT), Handler)

print "serving at port", PORT
httpd.serve_forever()

## Example with SSL support

* To generate key and cert files with OpenSSL use following command

`openssl req -x509 -newkey rsa:2048 -keyout key.pem -out cert.pem -days 365`

* To run secure HTTPs server create a following module

In [ ]:
!openssl req -x509 -newkey rsa:2048 -keyout key.pem -out cert.pem -days 365

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import ssl


httpd = HTTPServer(('localhost', 3333), BaseHTTPRequestHandler)

httpd.socket = ssl.wrap_socket (httpd.socket, 
        keyfile="key.pem", 
        certfile='cert.pem',
        server_side=True)

httpd.serve_forever()

## Advanced Python HTTP server

Let’s make our web server a little more advanced by handling requests

In [14]:
from http.server import HTTPServer, BaseHTTPRequestHandler


class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_GET(self):
# mandatory, otherwise the response wont be considered as valid. 
# We can check that it actually works by sending a request
        self.send_response(200)
# mandatory as well  
        self.end_headers()
        self.wfile.write(b'Hello, world!')


httpd = HTTPServer(('localhost', 3000), SimpleHTTPRequestHandler)
httpd.serve_forever()

127.0.0.1 - - [26/Aug/2019 13:37:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2019 13:37:22] "GET /favicon.ico HTTP/1.1" 200 -


KeyboardInterrupt: 

## `do_POST`

Let’s handle a POST request now

In [16]:
from http.server import HTTPServer, BaseHTTPRequestHandler

from io import BytesIO

In [17]:
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b'Hello, world!')

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        body = self.rfile.read(content_length)
        self.send_response(200)
        self.end_headers()
        response = BytesIO()
        response.write(b'This is POST request. ')
        response.write(b'Received: ')
        response.write(body)
        self.wfile.write(response.getvalue())

In [22]:
httpd = HTTPServer(('localhost', 3130), SimpleHTTPRequestHandler)
httpd.serve_forever()

127.0.0.1 - - [26/Aug/2019 13:40:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2019 13:40:42] "GET /favicon.ico HTTP/1.1" 200 -


KeyboardInterrupt: 

* The request body can be accessed via `self.rfile`. 

* It is a `BufferedReader` so `read([size])` method should be executed in order to get the contents. 
    * Note, that size should be explicitly passed to the function, otherwise the request will hang and never end.

* This is why obtaining `content_length` is necessary. 
    * It could be retrieved via `self.headers` and converted into an integer

## Other HTTP Server Alternatives
----
* Tornado
* Twisted
* Django
* Flask
* 